![banner](../logo/banner.png)

# 3.8 Sensores analogicos

Vamos a simular sensores analogicos usando un potenciomentro, el cual su variacion hara que realicemos una accion determinada.

## 3.8.1 Sensor de distancia infrarrojo

Un sensor infrarrojo la señal que nos entrega es la relacion de que tanta luz incide en el fototransistor, la cual es emitida por un led infrarrojo. Es decir, si la luz que emite el led llega al fototransistor debemos realizar o no una accion.

![irda](imgs/infrarrojo_analogico.jpg)

Ejemplo de un circuito de sensor infrarro simple:

![circuito irda](imgs/irda_circuito.png)

### 3.8.1 Aplicacion de sensores infrarrojos

**Ejemplo 1. **

## 3.8.2 Sensor de corriente 

Tenemos vario sensores de corrientes, aqui muestro el sensor de efecto Hall y una bobina.
Como tal el sensor hall detecta campos magneticos, no corriente, pero como un efecto del paso de la corriente genera un campo magnetico por ello se utiliza. 
Al pasar una corriente se genera un campo magnetico y si se le acerca una bobina se genera el efecto de induccion, lo que causa que en dicha bobina se genere una corriente equivalente a la corriente que pasa en el primer conductor.

En ambos casos podemos determinar la cantidad de corriente que esta pasando en dicho conductor gracias a su campo magnetico.

![hall](imgs/hall_analogico.jpeg)

![bobina](imgs/corriente_sensor.jpg)

### 3.8.2.1 Aplicacion de sensores de corriente

**Ejemplo 1. Necesitamos realizar un amperimetro, el sensor da un rango entre 0A hasta 1000mA. Se mandará el valor de corriente a la terminal**

![corriente simual](imgs/sensor_corriente_simul.png)

```python
from pyfirmata import Arduino, util
from time import sleep

# configuracion de la tarjeta

PORT = 'COM#'
board = Arduino(PORT)
print('Conexion lista')

util.Iterator(board).start()

# configuracion SENSOR
SENSOR_CURRENT_PIN = 0
current_sensor = board.get_pin(f'a:{str(SENSOR_CURRENT_PIN)}:i')
sleep(1)

while True:

    value = current_sensor.read()

    if value == None:
        continue

    current_value = value * 1000.0

    print(f'Value: {value}')
    print(f'La corriente es {current_value}mA')
    sleep(0.3)

```

**Ejemplo 2: Fusible electronico. Se necesita construir un fusible electronico ocupando un sensor de corriente, accionado un relay para proteger el aparato electronico que este conectado.**

![fusible simul](imgs/fusible_simul.png)

```python
from pyfirmata import Arduino, util
from time import sleep, time

def blink(board, pin, time):
    '''
    genera el blink en un led para indicar su estado
    '''
    board.digital[pin].write(1)
    sleep(time)
    board.digital[pin].write(0)
    sleep(time)

# configuracion de la tarjeta

PORT = '/tmp/ttyS1'
board = Arduino(PORT)
print('Conexion lista')

util.Iterator(board).start()

# configurando salidas
LED_PIN = 4
RELAY_PIN = 2

led = board.get_pin(f'd:{str(LED_PIN)}:o')
relay = board.get_pin(f'd:{str(RELAY_PIN)}:o')

# configuracion SENSOR
SENSOR_CURRENT_PIN = 0
current_sensor = board.get_pin(f'a:{str(SENSOR_CURRENT_PIN)}:i')
sleep(1)

# defino varibles auxiliares
ON = 1
OFF = 0

while True:
# vamos a simular 3 niveles del sensor
# apagado < 0.2
# normal 0.2 >= hasta 0.8
# overload (sobrecarga) >0.8

    value = current_sensor.read()

    if value == None:
        continue

    if value >= 0.8:
        relay.write(OFF)
        blink(board=board,pin=LED_PIN,time=0.2)

    elif value < 0.8 and value >= 0.2:
        relay.write(ON)
        led.write(ON)

    else:
        relay.write(ON)
        blink(board=board,pin=LED_PIN,time=0.5)

    sleep(2)
```

## 3.8.3 Sensor de temperatura

Existe una enorme variedad de sensores de temperatura, estaremos viendo el **[LM35](https://www.ti.com/lit/ds/symlink/lm35.pdf)**, sensor de temperatura lineal.

**Nos da una respuesta de 10mV/ºC**

![lm35](imgs/lm35.png)

---
**Ejemplo 1: Vamos a realizar un termometro con el LM35. La temperatura se mandará a la terminal. Debe mandar la temperatura en grados Celcius y grador Farenheits**

![termometro](imgs/termometro.png)

```python
from pyfirmata import Arduino, util
from time import sleep

# configuracion de la tarjeta

PORT = 'COM#'
board = Arduino(PORT)
print('Conexion lista')

util.Iterator(board).start()

# configuracion SENSOR
SENSOR_CURRENT_PIN = 0
current_sensor = board.get_pin(f'a:{str(SENSOR_CURRENT_PIN)}:i')
sleep(1)

while True:

    value = current_sensor.read()

    if value == None:
        continue

    celsius = value * 500

    farenheint = (celsius * 1.8) + 32
    print(f'Value: {value}')
    print(f'La temperatura Celsius {celsius}ºC')
    print(f'La temperatura Farenheit {farenheint}ºF')
    sleep(1)
```


**Ejemplo 2. Vamos a variar la velocidad de un ventialdor (motor DC) en funcion de la temperatura, simulando que debemos enfriar un tanque contenedor de un liquido. Cuando la temperatura sea menor a 20ºC, ventilador debe estar apagado, entre 20ºC y 50ºC debe trabajar a 1/3 de la potencia, cuando la temperatura este entre 50º y 90º, debe trabajar al 60% el ventilador, cuando supere la temperatura de 90ºC debe trabaja a su maxima potencia el ventilador. Cuando este en la temperatura este por encima de 90ºC debe parpadear rapido un led , cuando este por debajo solo queda encendido y cuando este apagado el ventilador el led parpadea lento.**

![](imgs/temp_ventilador.png)

```python
from pyfirmata import Arduino, util
from time import sleep
import math

def blink(board, pin, time):
    board.digital[pin].write(1)
    sleep(time)
    board.digital[pin].write(0)
    sleep(time)

# configuracion de la tarjeta
PORT = '/tmp/ttyS1'
board = Arduino(PORT)
print('Conexion lista')

util.Iterator(board).start()

# configuro la salida del PWM para el ventilador
MOTOR_PIN = 3
motor = board.get_pin(f'd:{str(MOTOR_PIN)}:p')

# configuro salida del LED
LED_PIN = 8
led = board.get_pin(f'd:{str(LED_PIN)}:o')

# configuracion SENSOR
SENSOR_CURRENT_PIN = 0
current_sensor = board.get_pin(f'a:{str(SENSOR_CURRENT_PIN)}:i')
sleep(1)


while True:

    value = current_sensor.read()

    if value == None:
        continue

    celsius = value * 500
    celsius = math.ceil(celsius) # hacemos un redondeo para evitar problemas en las comparaciones

    if celsius <= 50:
        blink(board=board, pin=LED_PIN,time=1)
        motor.write(0.00)
    elif celsius > 50 and celsius <= 100:
        motor.write(0.33)
        led.write(1)
    elif celsius > 100 and celsius <= 150:
        motor.write(0.6)
        led.write(1)
    elif celsius > 150:
        motor.write(0.999)
        blink(board=board,pin=LED_PIN,time=0.2)

    print(f'temperatura {celsius}')

```


## 3.8.4 Sensor de luz (LDR)

Para el sensor de luz ultilizaremos el divisor de tension con una LDR.

![ldr div](imgs/circuitos_ldr.png)


**Ejemplo 1: En funcion de la luz que incide en el sensor se manda a la terminal el porcentaje de luz que incide en él. Simulando que se encuentra en una habitación.**

![ldr led](imgs/ldr_led.png)

**Ejemplo 2: En funcion de la luz que incide en el sensor, vamos a mover un servomotor que estaria abriendo las cortinas de la ventana con algun mecanismo.**



---

## Practicas - sensores Analogicos

1. Simular un sensor de distancia ultrasonico, que cuando identifique que el objeto esta a una distancia menor a 10cm haga parpadear una barra de leds.
2. Simular un sensor de luz junto con un sensor de presencia (digital). Dicho sensores trabajan a la par, si hay poca luz y existe presencia en el cuarto se debe encender un foco. En caso que no exista presencia sin importar la intensidad de la luz, el foco no enciende.
3. Simular un sensor infrarrojo para accionar el movimiento de un servomotor, entre mas luz incida debe incrementar el angulo del servomotor. Es decir, entre mas cerca se incrementa el angulo, entre mas lejos es menor el angulo.

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)